In [1]:
from tesi.zappai.utils.genetic import GeneticAlgorithm
from tesi.zappai.di import (
    get_crop_optimizer_service,
    get_session_maker,
    get_cds_api,
    get_location_repository,
    get_past_climate_data_repository,
    get_future_climate_data_repository,
    get_climate_generative_model_repository,
    get_crop_repository,
    get_crop_yield_data_repository,
    get_crop_yield_model_service,
)

print("Before everything")

session_maker = get_session_maker()
cds_api = get_cds_api()
location_repository = get_location_repository(session_maker=session_maker)
past_climate_data_repository = get_past_climate_data_repository(
    session_maker=session_maker,
    cds_api=cds_api,
    location_repository=location_repository,
)
future_climate_data_repository = get_future_climate_data_repository(
    session_maker=session_maker, cds_api=cds_api
)
climate_generative_model_repository = get_climate_generative_model_repository(
    session_maker=session_maker,
    location_repository=location_repository,
    past_climate_data_repository=past_climate_data_repository,
    future_climate_data_repository=future_climate_data_repository,
)
crop_repository = get_crop_repository(session_maker=session_maker)
crop_yield_data_repository = get_crop_yield_data_repository(
    crop_repository=crop_repository,
    location_repository=location_repository,
    past_climate_data_repository=past_climate_data_repository,
)
crop_yield_model_service = get_crop_yield_model_service(
    past_climate_data_repository=past_climate_data_repository,
    location_repository=location_repository,
    crop_yield_data_repository=crop_yield_data_repository,
    crop_repository=crop_repository,
)
crop_optimizer_service = get_crop_optimizer_service(
    past_climate_data_repository=past_climate_data_repository,
    location_repository=location_repository,
    climate_generative_model_repository=climate_generative_model_repository,
    crop_repository=crop_repository,
    future_climate_data_repository=future_climate_data_repository
)

2024-07-19 13:33:05.559251: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:33:05.561906: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 13:33:05.595546: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-19 13:33:06.244249: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Before everything


In [2]:
import pandas as pd
from typing import cast
from tesi.zappai.di import get_crop_optimizer_service
from tesi.zappai.dtos import CropDTO
from tesi.zappai.utils.common import enrich_data_frame_with_stats
from tesi.zappai.utils.genetic import (
    Individual,
    Population,
    individual_to_int,
)
from tesi.zappai.services.crop_yield_model_service import (
    FEATURES as CROP_YIELD_MODEL_FEATURES,
)
from sklearn.ensemble import RandomForestRegressor
from tesi.zappai.utils.common import calc_months_delta
from concurrent.futures import ProcessPoolExecutor


location = await location_repository.get_location_by_country_and_name(
    "Italy", "Policoro"
)

if location is None:
    raise Exception()

crop = await crop_repository.get_crop_by_name("maize")
if crop is None:
    raise Exception()

result = await crop_optimizer_service.get_best_crop_sowing_and_harvesting(crop_id=crop.id, location_id=location.id)

best_combination = result.best_combinations[-1]
print(f"Sowing: {best_combination.sowing_year}/{best_combination.sowing_month}")
print(f"Harvesting: {best_combination.harvest_year}/{best_combination.harvest_month}")
print(f"Duration: {best_combination.duration}")
print(f"Estimated yield per hectar: {best_combination.estimated_yield_per_hectar}")

2024-07-19 13:33:07.161158: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-19 13:33:07.161662: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Sowing: 2024/9
Harvesting: 2025/11
Duration: 14
Estimated yield per hectar: 8635.79017996543
